In [7]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from pmdarima import AutoARIMA

import warnings
warnings.filterwarnings("ignore")

In [8]:
df = pd.read_csv('../air-traffic-prediction/AirtrafficA4.csv')
df = df.dropna()
df['DATE'] = df['YEAR'].astype(str) + '-' + df['MONTH']
df['TOTAL DEPARTURES'] = df['TOTAL DEPARTURES'].str.replace(',', '').astype(float)
df['HOURS FLOWN'] = q = df['HOURS FLOWN'].str.replace(',', '').astype(float)
df['KILOMETRE FLOWN'] = df['KILOMETRE FLOWN'].str.replace(',', '').astype(float)
df['PASSENGERS CARRIED'] = df['PASSENGERS CARRIED'].str.replace(',', '').astype(float)
df['AVAILABLE SEAT KILOMETRE (IN THOUSAND)'] = df['AVAILABLE SEAT KILOMETRE (IN THOUSAND)'].str.replace(',', '').astype(float)
df[' FREIGHT CARRIED (IN TONNE)'] = df[' FREIGHT CARRIED (IN TONNE)'].str.replace(',', '').astype(float)
df['MAIL CARRIED (IN TONNE)'] = df['MAIL CARRIED (IN TONNE)'].str.replace(',', '').astype(float)
df.rename(columns={' FREIGHT CARRIED (IN TONNE)': 'FREIGHT CARRIED (IN TONNE)'}, inplace=True)

df['MONTH'] = df['MONTH'].str.strip()
df['MONTH'] = df['MONTH'].map({'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUNE': 6, 'JULY': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12})

#sorting df on the basis of year
df = df.sort_values(by=['YEAR', 'MONTH'])

# df.tail()

### Q2_1 Part C
Train a Global Model say prophet or any other. These models are fit once on related
time series and used later for prediction, instead of fit being called for every time series separately.
You can use all of the other columns present in the dataset to create related Time-series.
Again Only report evaluations by this method in the report.

In [9]:
from prophet import Prophet

prophet_model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
prophet_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
for regressor in ['TOTAL DEPARTURES', 'HOURS FLOWN', 'KILOMETRE FLOWN', 'AVAILABLE SEAT KILOMETRE (IN THOUSAND)', 
                  'FREIGHT CARRIED (IN TONNE)', 'MAIL CARRIED (IN TONNE)']:
    prophet_model.add_regressor(regressor)
    
prophet_model.fit(df.rename(columns={'DATE': 'ds', 'PASSENGERS CARRIED': 'y'}))

future = prophet_model.make_future_dataframe(periods=12, freq='MS')
for regressor in ['TOTAL DEPARTURES', 'HOURS FLOWN', 'KILOMETRE FLOWN', 'AVAILABLE SEAT KILOMETRE (IN THOUSAND)', 
                  'FREIGHT CARRIED (IN TONNE)', 'MAIL CARRIED (IN TONNE)']:
    future[regressor] = df[regressor].iloc[-1]

prophet_forecast = prophet_model.predict(future)

prophet_forecast.rename(columns={'ds': 'YEAR_MONTH', 'yhat': 'PASSENGERS CARRIED'}, inplace=True)
prophet_forecast['YEAR_MONTH'] = prophet_forecast['YEAR_MONTH'].dt.strftime('%Y %b').str.upper()

prophet_forecast[['YEAR_MONTH', 'PASSENGERS CARRIED']].tail(12).to_csv('Q2c_forecast_prophet_regressors.csv', index=False)
display(prophet_forecast[['YEAR_MONTH', 'PASSENGERS CARRIED']].tail(12))

20:47:04 - cmdstanpy - INFO - Chain [1] start processing
20:47:04 - cmdstanpy - INFO - Chain [1] done processing


,YEAR_MONTH,PASSENGERS CARRIED
126,2023 SEP,7.823192e+06
127,2023 OCT,7.917891e+06
128,2023 NOV,8.043140e+06
129,2023 DEC,8.318005e+06
130,2024 JAN,7.981118e+06
131,2024 FEB,8.120254e+06
132,2024 MAR,8.334904e+06
133,2024 APR,8.072358e+06
134,2024 MAY,8.602720e+06
135,2024 JUN,8.177198e+06
